In [2]:
import random
hello_dict = { 0 : "hello!", 1 : "I'm here", 2 : "hey!" ,3 : "what's up?" }
Goodbye_dict = { 0 : "Goodbye!", 1 : "Nice to see you", 2 : "ok!" ,3 : "Well.." }
num = random.randrange(4)
hello_dict[num]

'hey!'

In [35]:
import json
a = "www"
c = "qqqq"
# Opening JSON file
with open(r'..\docs\user.json') as f:
    data = json.load(f)
"U39ef08913a1aa5a8e5f100399fe9fd2c"in data
# data[a] = c
# print(data)
# with open(r'D:\linebot_self\docs\user.json', 'w') as f:
#     json.dump(data, f,ensure_ascii=False)


True

In [38]:
import requests
from datetime import datetime,timedelta
def weather(WEATHER_API_KEY,user):

    now = datetime.now().date()
    weather_apikey = WEATHER_API_KEY
    weather_apikey = 'CWB-B64CD8B7-02BF-441E-B253-C654F478E513'
    dataid = 'F-D0047-069' #新北市
    element_name = "T,Wx,PoP6h,AT"

    url = f'https://opendata.cwb.gov.tw/api/v1/rest/datastore/{dataid}?Authorization={weather_apikey}&locationName={user[2:5]}&elementName={element_name}'
    response = requests.get(url)
    data = response.json()

    date_code = {'今天':now,'明天':now + timedelta(days=1)}
    
    extra = data['records']['locations'][0]['location'][0]['weatherElement']

    relpy_text = user[2:] +"\n"

    for j in range(len(extra[0]['time'])):
        # 讀取資料第一筆時間
        moment = datetime.strptime(extra[0]['time'][j]['startTime'], '%Y-%m-%d %H:%M:%S')
        
        # 判斷所需的日期天氣資料

        if str(date_code[user[:2]]) in str(moment):

            relpy_text+= str(moment) 
            relpy_text+='\n'
            for i in range(len(extra)-1):
                description = extra[i]['description']
                try:
                    wx = extra[i]['time'][j]['elementValue'][0]['value']
                except IndexError:
                    wx = '尚未預測'
                relpy_text+=description+":"+ wx 
                relpy_text+='\n'

            try:
                relpy_text+=extra[-1]['description']+":"+extra[-1]['time'][j//2]['elementValue'][0]['value']+"%"
                relpy_text+='\n'
                relpy_text+='\n'
            except IndexError:
                relpy_text+='N\A'
                
    return relpy_text
print(weather(1,"明天板橋區天氣"))

板橋區天氣
2023-03-20 00:00:00
天氣現象:多雲
體感溫度:19
溫度:19
6小時降雨機率:20%

2023-03-20 03:00:00
天氣現象:多雲
體感溫度:20
溫度:19
6小時降雨機率:20%

2023-03-20 06:00:00
天氣現象:多雲
體感溫度:20
溫度:18
6小時降雨機率:0%

2023-03-20 09:00:00
天氣現象:多雲
體感溫度:23
溫度:21
6小時降雨機率:0%

2023-03-20 12:00:00
天氣現象:多雲
體感溫度:28
溫度:26
6小時降雨機率:30%

2023-03-20 15:00:00
天氣現象:短暫雨
體感溫度:27
溫度:26
6小時降雨機率:30%

2023-03-20 18:00:00
天氣現象:晴
體感溫度:25
溫度:23
6小時降雨機率:0%

2023-03-20 21:00:00
天氣現象:晴
體感溫度:24
溫度:22
6小時降雨機率:0%




In [46]:
a = "明天板橋區天氣\n\n"
a[:-4]

'明天板橋區'

In [43]:
import time
import js2py
import pandas as pd
def now_weather(name):
  timestr = time.strftime("%Y%m%d%H")
  url = 'https://www.cwb.gov.tw/Data/js/GT/TableData_GT_T_63.js?T='+timestr+'-4&_=1657432140292'
  headers = {"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.64 Safari/537.36 Edg/101.0.1210.47"}
  download = requests.get(url,headers = headers)
  download = download.text
  cityID = [6301200,6301100,6300600,6300400,6300100,6301000,6300700,6300500,6300300,6300200,6300900,6300800]
  cityname = {"萬華區":6,"中正": 7,"南港" : 10,"文山": 11}
  aid = cityname[name]
  text = js2py.eval_js(download)
  C_AT=[]
  C_T=[]
  RH=[]
  Rain=[]
  Sunrise=[]
  Sunset=[]
  for item in cityID:
    C_AT.append(text[item]["C_AT"])
    C_T.append(text[item]["C_T"])
    Rain.append(text[item]["Rain"])
    RH.append(text[item]["RH"])
    Sunrise.append(text[item]["Sunrise"])
    Sunset.append(text[item]["Sunset"])
    

  output=[cityID,C_AT,C_T,RH,Rain,Sunrise,Sunset]
  df=pd.DataFrame(output)
  dt=df.T
  dt.columns=['Cityid','C_AT','C_T','RH','Rain','Sunrise','Sunset']
  timestr = time.strftime("%Y%m%d")
  return '\n'+timestr + '\n'+name+'降雨機率為 ' + dt['Rain'][aid]  +"%\n最高溫: "+ dt['C_AT'][aid] + " 度\n最低溫: " + dt['C_T'][aid] +'度'
# 6300700 萬華 6
#  63000500 中正 7
#6300900 南港 10
#6300800 文山 11
print(now_weather())


20230319
台北市中正區降雨機率為 0.0%
最高溫: 23 度
最低溫: 21度


In [2]:
import requests
import os
def notify(message):
    
    # LINE Notify 權杖
    token = os.getenv("line_notify_token")

    # HTTP 標頭參數與資料
    token = 'q4AiBIuqOz1zhUoTUOnzJEDvjObU5zq4evxx1Shxqm1'
    headers = { "Authorization": "Bearer " + token }
    data = {'message':'\n'+message}

    # 以 requests 發送 POST 請求
    requests.post("https://notify-api.line.me/api/notify",headers = headers, data = data)
    # headers = headers, data = data, files = files)
if __name__ == '__main__':
    print(notify("我是卓子揚"))

None


In [11]:
from linebot.models import QuickReply, QuickReplyButton,PostbackAction
location = ["宜蘭縣","花蓮縣","臺東縣","澎湖縣","金門縣","連江縣","臺北市","新北市","桃園市","臺中市","臺南市","高雄市","基隆市","新竹縣","新竹市","苗栗縣","彰化縣","南投縣","雲林縣","嘉義縣","嘉義市","屏東縣"]
item = []
count = 0
for items in location:
    count+=1
    item.append(QuickReplyButton( action=PostbackAction ( label=items, data="C" + items ) ))
quick_reply=QuickReply(item)
quick_reply

{"items": [{"action": {"data": "C\u5b9c\u862d\u7e23", "label": "\u5b9c\u862d\u7e23", "type": "postback"}, "type": "action"}, {"action": {"data": "C\u82b1\u84ee\u7e23", "label": "\u82b1\u84ee\u7e23", "type": "postback"}, "type": "action"}, {"action": {"data": "C\u81fa\u6771\u7e23", "label": "\u81fa\u6771\u7e23", "type": "postback"}, "type": "action"}, {"action": {"data": "C\u6f8e\u6e56\u7e23", "label": "\u6f8e\u6e56\u7e23", "type": "postback"}, "type": "action"}, {"action": {"data": "C\u91d1\u9580\u7e23", "label": "\u91d1\u9580\u7e23", "type": "postback"}, "type": "action"}, {"action": {"data": "C\u9023\u6c5f\u7e23", "label": "\u9023\u6c5f\u7e23", "type": "postback"}, "type": "action"}, {"action": {"data": "C\u81fa\u5317\u5e02", "label": "\u81fa\u5317\u5e02", "type": "postback"}, "type": "action"}, {"action": {"data": "C\u65b0\u5317\u5e02", "label": "\u65b0\u5317\u5e02", "type": "postback"}, "type": "action"}, {"action": {"data": "C\u6843\u5712\u5e02", "label": "\u6843\u5712\u5e02", "ty

In [12]:
count

22